In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np
import os
import re
import lxml

In [2]:
driver = webdriver.Chrome("C:\\Users\\bobby\\Desktop\\Python_project\\chromedriver.exe")

In [3]:
driver.get("https://www.ufc.com/athletes/all")

In [4]:
ele=driver.find_element_by_id("onetrust-accept-btn-handler")

In [5]:
ele.click()

In [6]:
time.sleep(2)

In [7]:
total=BeautifulSoup(driver.page_source,"lxml")

In [8]:
totalAth=total.find_all("div", {"class": "althelete-total"})

In [9]:
stotal=re.search("\d+",str(totalAth)).group(0)
stotal=int(int(stotal)/11)


In [10]:
#for i in range(stotal):
#    ele=driver.find_element_by_class_name("pager__item")
#    ele.click()
#    time.sleep(10)

In [11]:
total=BeautifulSoup(driver.page_source,"lxml")

In [12]:
links=total.find_all("a", {"class": "e-button--black"})

In [13]:
ufc = "https://www.ufc.com"
links[0]["href"]
linksAH=pd.DataFrame(np.nan, index=range(len(links)), columns=["link"],dtype="string")

In [14]:
for i in range(len(links)):
        linksAH.at[i,'link']= ufc +str(links[i]["href"])
        

In [15]:
driver.get(linksAH.iloc[0,0])

In [16]:
time.sleep(2)

In [17]:
ele = driver.find_element_by_class_name("e-button--white")
ele.click()

while True:
    try:
        ele = driver.find_element_by_class_name("pager__item")
        ele.click()
        time.sleep(2)
    except:
        break

In [18]:
time.sleep(2)
stats = BeautifulSoup(driver.page_source,"lxml")
columns = ["name","str.ac","grap.ac","sig.str.land","sig.str.att","take.land","take.att",
           "sig.str.land.min","sig.str.abs.min","take.avg.15min","sub.avg.15min","sig.str.def",
          "take.def","knock.ratio","avg.time","sig.str.head","sig.str.head.percent","sig.str.body",
          "sig.str.body.percent","sig.str.leg","sig.str.leg.percent","sig.str.position.stand",
          "sig.str.position.stand","sig.str.position.clinch","sig.str.position.ground",
          "sig.str.position.stand.percent","sig.str.position.clinch.percent","sig.str.position.ground.percent"]
StatsOfFigthers=pd.DataFrame(columns =columns)
temp = pd.DataFrame(columns =columns,index =range(1))

In [19]:
#for a in stats.find_all(attrs={"class":"c-record__promoted"}):
#    try:
#        print(re.search("\d+",str(a)).group(0))
#    except:
#            print("0")      

In [20]:
#find striking acc and grappling acc
for a in stats.find_all(attrs ={"class":"c-overlap-athlete-detail__card"}):
    title =re.sub("<.*?>","",str(a.title))
    title =re.sub("\d+%","",title)
    title = title.strip()
    title = title.lower()
    if title == "striking accuracy":
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"}))))
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0])))
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1])))
    if title == "grappling accuracy":
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"}))))
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0])))
        print(re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1])))

#for a in stats.find_all(attrs={"class":"e-chart-circle__percent"}):
#    print(a)
#    print(re.sub("<.*?>","",str(a)))

[45%]
210
469
[24%]
5
29


In [60]:

for a in stats.find_all(attrs={"class":"c-stat-compare__group-1"}):
    label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
    num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))

for a in stats.find_all(attrs={"class":"c-stat-compare__group-2"}):
    label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
    num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))
    


<div class="c-stat-compare__group-2">
<div class="c-stat-compare__number">2.45
          </div>
<div class="c-stat-compare__label">Sig. Str. Absorbed</div>
<div class="c-stat-compare__label-suffix">Per Min</div>
</div>
<div class="c-stat-compare__group-2">
<div class="c-stat-compare__number">0.18
          </div>
<div class="c-stat-compare__label">Submission avg</div>
<div class="c-stat-compare__label-suffix">Per 15 Min</div>
</div>
<div class="c-stat-compare__group-2">
<div class="c-stat-compare__number">47
                      <div class="c-stat-compare__percent">%</div></div>
<div class="c-stat-compare__label">Takedown Defense</div>
</div>
<div class="c-stat-compare__group-2">
<div class="c-stat-compare__number">10:43
          </div>
<div class="c-stat-compare__label">Average fight time</div>
</div>


In [66]:
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_value"}))))
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_percent"}))))
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_value"}))))
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_percent"}))))
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_value"}))))
print(re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_percent"}))))


131
62%
37 
18%
42
20%


In [67]:
for a in stats.find_all(attrs={"class":"c-bio__field"}):
    lab=a.find(attrs={"class":"c-bio__label"})
    lab = re.sub("<.*?>","",str(lab))
    c=re.sub("<.*?>","",str(a))
    c= re.search("\d+",str(c))
    if(lab=="Age"):
        print(c.group(0))
    if(lab=="Height"):
        print(c.group(0))
    if(lab=="Reach"):
        print(c.group(0))
    if(lab=="Leg reach"):
        print(c.group(0))

39
75
76
41


In [ ]:
listofights

In [70]:
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__matchup"}):
    #if("Win"==re.sub("<.*?>","",str(a)).strip()):
    for j in a:
        listofights.append(j)
    

In [72]:
for i in range(len(listofights)):
    for a in range(len(listofights)):
        try:
            if str(listofights[a]) == "\n":
                listofights.pop(a)
        except:
            break


In [74]:
listoffightnames =[]
for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__headline"}):
    listoffightnames.append(re.sub("<.*?>","",str(a)))

In [75]:
name=re.search("-.*",str(linksAH.iloc[0,0])).group(0)
name=re.sub("-","",name)

In [76]:
name

'abdurakhimov'

In [77]:
listoffightnames

['Blaydes vs Abdurakhimov',
 'Tybura vs Abdurakhimov',
 'Arlovski vs Abdurakhimov',
 'Sherman vs Abdurakhimov',
 'Lewis vs Abdurakhimov',
 'Abdurakhimov vs Harris',
 'Hamilton vs Abdurakhimov',
 'Abdurakhimov vs Johnson']

In [78]:
re.sub("vs","",listoffightnames[0])

'Blaydes  Abdurakhimov'

In [79]:
leftname=re.search(".*? +",listoffightnames[0]).group(0)
rightname=re.search(" +.*",listoffightnames[0]).group(0)

In [80]:
leftname=leftname.strip()
leftname=leftname.lower()
rightname=rightname.strip()
rightname=rightname.lower()
name=name.strip()
name=name.lower()